<a href="https://colab.research.google.com/github/mallikharjun07/ML/blob/master/SMS%20Spam%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Clone the entire repo.
!git clone -l -s git://github.com/mallikharjun07/ML cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 2.03 MiB | 4.32 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/cloned-repo/cloned-repo
 bbc_news_mixed.csv   README.md  'SMS Spam Classification.ipynb'   spamdata.csv


In [0]:
file = '/content/cloned-repo/cloned-repo/spamdata.csv'

In [6]:
import pandas as pd
data = pd.read_csv(file)
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
from sklearn.preprocessing import LabelEncoder
lencod = LabelEncoder()
target = lencod.fit_transform(data['label'])

In [16]:
target[:5]

array([0, 0, 1, 0, 0])

# Load keras libraries

In [0]:

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [19]:
token = Tokenizer()
token.fit_on_texts(data["text"])
word_index = token.word_index
# type(word_index) - it is dictionary

dict

In [0]:
trainx, valx, trainy, valy = train_test_split(data["text"], target)

train_seq_x = pad_sequences(token.texts_to_sequences(trainx),maxlen=70)
val_seq_x = pad_sequences(token.texts_to_sequences(valx),maxlen=70)

In [25]:
train_seq_x[:100]

array([[   0,    0,    0, ...,    3,  224,   12],
       [   0,    0,    0, ...,   47, 8857, 2985],
       [   0,    0,    0, ...,   17,    2,  414],
       ...,
       [   0,    0,    0, ...,  311,   29,  283],
       [   0,    0,    0, ..., 3209,    7,  536],
       [   0,    0,    0, ...,   11, 1315, 1413]], dtype=int32)

In [0]:
from keras import layers, models, optimizers
def create_cnn():
    input_layer = layers.Input((70,))
    embedding_layer = layers.Embedding(len(word_index))+1,300)(input_layer)
    conv_layer = layers.Convolution1D(100,3,activation='relu')(embedding_layer)
    pooling_layer = layers.Global1MaxPool1D()(conv_layer)

    output_layer = layers.Dense(50,activation='relu')(pooling_layer)

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_val, valid_y):
    classifier.fit(feature_vector_train, label)
    predictions = classifier.predict(feature_vector_val)
    predictions = predictions.argmax(axis=-1)
    return accuracy_score(predictions, valid_y)

In [0]:
from keras import layers, models, optimizers 

def create_cnn():
    
    input_layer = layers.Input((70, ))
    
    embedding_layer = layers.Embedding(len(word_index)+1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)
    
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    output_layer = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer = layers.Dropout(0.25)(output_layer)    
    output_layer = layers.Dense(1, activation="sigmoid")(output_layer)
    
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=optimizers.Adam(), loss="binary_crossentropy")
    return model

In [0]:
classifier = create_cnn()
train_model(classifier, trainx, trainy, valx, valy)

In [0]:
## feature engineering 

## meta features
data["cleaned"] = data["cleaned"].fillna("")

data["digit_count"] = data["text"].apply(lambda x : sum([1 if w.isdigit() else 0 for w in x.split()]))
data["upper_count"] = data["text"].apply(lambda x : sum([1 if w.isupper() else 0 for w in x.split()]))
data["word_count"] = data["cleaned"].apply(lambda x: len(x.split()))
data["char_count"] = data["cleaned"].apply(lambda x: len(x))
data["char_nospace_count"] = data["cleaned"].apply(lambda x: len(x.replace(" ","")))

data

In [0]:
## nlp based features 

pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

def pos_check(x, flag):
    tags = nltk.pos_tag(nltk.word_tokenize(x))
    count = 0
    for tag in tags:
        tag = tag[1]
        if tag in pos_dic[flag]:
            count += 1
    return count

data['noun_count'] = data['cleaned'].apply(lambda x: pos_check(x, 'noun'))
data['verb_count'] = data['cleaned'].apply(lambda x: pos_check(x, 'verb'))
data

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cvz = CountVectorizer(analyzer='word') 
cvz.fit(data["cleaned"].values)
count_vectors = cvz.transform(data["cleaned"].values)

In [0]:
count_vectors

In [0]:
word_tfidf = TfidfVectorizer(analyzer='word') 
word_tfidf.fit(data["cleaned"].values)
word_vectors_tfidf = word_tfidf.transform(data["cleaned"].values)

ngram_tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,3)) 
ngram_tfidf.fit(data["cleaned"].values)
ngarm_vectors_tfidf = ngram_tfidf.transform(data["cleaned"].values)

char_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(1,3)) 
char_tfidf.fit(data["cleaned"].values)
char_vectors_tfidf = char_tfidf.transform(data["cleaned"].values)

In [0]:
tfidf = dict(zip(word_tfidf.get_feature_names(), word_tfidf.idf_))
tfidf = pd.DataFrame(columns=['title_word_tfidf']).from_dict(dict(tfidf), orient='index')
tfidf.columns = ['title_word_tfidf']
tfidf.sort_values(by=['title_word_tfidf'], ascending=False).head()

In [0]:
tfidf.sort_values(by=['title_word_tfidf'], ascending=False).tail()

In [0]:
from scipy.sparse import hstack, csr_matrix

meta_features = ['digit_count', 'upper_count', 'word_count', 'char_count', 'char_nospace_count', 'noun_count', 'verb_count']
feature_set1 = data[meta_features]
train = hstack([word_vectors_tfidf, csr_matrix(feature_set1)], 'csr')

In [0]:
train

In [0]:
from sklearn.preprocessing import LabelEncoder

target = data['label'].values
target = LabelEncoder().fit_transform(target)

In [0]:
from sklearn.model_selection import train_test_split
trainx, valx, trainy, valy = train_test_split(train, target)

In [0]:
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import ensemble
# import xgboost
from sklearn.metrics import accuracy_score

In [0]:
## NaiveBayes
model = naive_bayes.MultinomialNB()
model.fit(trainx, trainy)
preds = model.predict(valx)
accuracy_score(preds, valy)

In [0]:
model = LogisticRegression()
model.fit(trainx, trainy)
preds = model.predict(valx)
accuracy_score(preds, valy)

In [0]:
model = svm.SVC()
model.fit(trainx, trainy)
preds = model.predict(valx)
accuracy_score(preds, valy)

In [0]:
model = ensemble.ExtraTreesClassifier()
model.fit(trainx, trainy)
preds = model.predict(valx)
accuracy_score(preds, valy)

In [0]:
# load the pre-trained word-embedding vectors
import numpy as np 
embeddings_index = {}
for i, line in enumerate(open('pretrained.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

In [0]:
# create a tokenizer 
from keras.preprocessing import text, sequence
from sklearn import model_selection 

token = text.Tokenizer()
token.fit_on_texts(data['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data['text'], target)
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
train_x.shape

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [0]:
from keras import layers , models , optimizers
from sklearn import metrics

def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)